## Definitions

### Observables
+ time <span style="background-color: #F5F5F5">$t_{k}$</span>
+ position <span style="background-color: #F5F5F5">$m_{k}$</span>
+ velocity <span style="background-color: #F5F5F5">$v_{k}$</span>
+ LFP power <span style="background-color: #F5F5F5">$y_{k}$</span>
+ population spiking activity of $C$ cells <span style="background-color: #F5F5F5">$\Delta N_{k}^{1:C}$</span>


### (Semi-)Latent Variables
+ replay indicator
$
  \begin{equation}
    I_{k} =
    \begin{cases}
      0, & \text{if replay at}\ t_{k} \\
      1, & \text{if no replay at}\ t_{k} \\
    \end{cases}
  \end{equation}
$
+ position/replay position 
$
  \begin{equation}
    x_{k} =
    \begin{cases}
      \text{unobserved replay position}, & \text{if}\ I_{k} = 1 \\
      m_{k}, & \text{if}\ I_{k} = 0 \\
    \end{cases}
  \end{equation}
$

### State Transitions
+ replay state transition $\Pr(I_{k} \mid I_{k-1}, v_{k-1})$
+ movement state transition 
$
  \begin{equation}
    p(x_{k} \mid x_{k-1}, I_{k}, I_{k-1}) =
    \begin{cases}
      \delta (m_{k}), & \text{if}\ I_{k} = 0 \\
      U(a, b), & \text{if}\ I_{k} = 1 \text{ and }\ I_{k-1} = 0 \\
      N(x_{k-1}, \sigma), & \text{if}\ I_{k} = 1 \text{ and }\ I_{k-1} = 1 \\
    \end{cases}
  \end{equation}
$

### Likelihoods
+ velocity likelihood $p(v_{k} \mid I_{k}, v_{k})$
+ LFP power likelihood $p(y_{k} \mid I_{k}, y_{k})$
+ population spiking activity likelihood $p(\Delta N_{k}^{1:C} \mid I_{k}, )$

### Joint Filter Distribution
+ $p(I_{k}, x_{k} \mid y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k})$


### Calculation Goals
+ replay probability $\Pr(I_{k} \mid y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k})$
+ replay position $p(x_{k} \mid I_{k} = 1, y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k})$


## Filter Derivation


1. Bayes ($ posterior \propto likelihood * prior $)
$$
p(I_{k}, x_{k} \mid y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k}) \propto
p(y_{k} \mid I_{k}, x_{k}, y_{1:k-1}, \Delta N_{1:k}^{1:C}, v_{1:k}) *
p(\Delta N_{k}^{1:C} \mid I_{k}, x_{k}, y_{1:k}, \Delta N_{1:k-1}^{1:C}, v_{1:k}) * 
p(v_{k} \mid I_{k}, x_{k}, y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) *
p(I_{k}, x_{k} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1})
$$

2. Simplify likelihoods
$$
p(I_{k}, x_{k} \mid y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k})  \propto 
p(y_{k} \mid I_{k}, y_{1:k-1}) *
p(\Delta N_{k}^{1:C} \mid x_{k}, \Delta N_{1:k-1}^{1:C}) *
p(v_{k} \mid I_{k}, v_{1:k-1}) *
p(I_{k}, x_{k} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1})
$$

3. Chapman-Kolmogorov to define prior in terms of previous time step's posterior
$$
\begin{align*}
p(I_{k}, x_{k} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) &= \sum_{I_{k-1}} \int p(I_{k}, I_{k-1}, x_{k}, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1}
\\ &= \sum_{I_{k-1}} \int p(I_{k}, x_{k} \mid I_{k-1}, x_{k-1}, y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * p(I_{k-1}, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1} \tag{Markov}
\\ &= \sum_{I_{k-1}} \int p(I_{k}, x_{k} \mid I_{k-1}, x_{k-1}) * p(I_{k-1}, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1} \tag{Markov}
\\ &= \sum_{I_{k-1}} \int p(x_{k} \mid x_{k-1}, I_{k}, I_{k-1}) * \Pr(I_{k} \mid I_{k-1}, v_{k-1}) *
p(I_{k-1}, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1}
\end{align*}$$

is the last step just another simplification?

4. Final Filter
$$
p(I_{k}, x_{k} \mid y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k})  \propto 
p(y_{k} \mid I_{k}, y_{1:k-1}) *
p(\Delta N_{k}^{1:C} \mid x_{k}, \Delta N_{1:k-1}^{1:C}) *
p(v_{k} \mid I_{k}, v_{1:k-1}) *
\sum_{I_{k-1}} \int p(x_{k} \mid x_{k-1}, I_{k}, I_{k-1}) * \Pr(I_{k} \mid I_{k-1}, v_{k-1}) *
p(I_{k-1}, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1}
$$

## Computing goals from filter

+ replay position
$$
\begin{align*}
p(x_{k} \mid I_{k} = 1, y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k}) &= \frac{p(x_{k}, I_{k} = 1 \mid y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k})}{\Pr(I_{k} = 1 \mid y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k})}  \tag{Def. of conditional probability}
\\
\\ p(x_{k}, I_{k} = 1 \mid y_{1:k}, \Delta N_{1:k}^{1:C}, v_{1:k}) &\propto p(y_{k} \mid I_{k} = 1, y_{1:k-1}) *
p(\Delta N_{k}^{1:C} \mid x_{k}, \Delta N_{1:k-1}^{1:C}) *
p(v_{k} \mid I_{k} = 1, v_{1:k-1}) *
\sum_{I_{k-1}} \int p(x_{k} \mid x_{k-1}, I_{k} = 1, I_{k-1}) * \Pr(I_{k} = 1 \mid I_{k-1}, v_{k-1}) *
p(I_{k-1}, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1}
\\ &\propto \text{likelihoods} * \Bigg(
\int p(x_{k} \mid x_{k-1}, I_{k} = 1, I_{k-1} = 0) * \Pr(I_{k} = 1 \mid I_{k-1} = 0, v_{k-1}) *
p(I_{k-1} = 0, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1} +
\int p(x_{k} \mid x_{k-1}, I_{k} = 1, I_{k-1} = 1) * \Pr(I_{k} = 1 \mid I_{k-1} = 1, v_{k-1}) *
p(I_{k-1} = 1, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1} \Bigg)
\\ &\propto \text{likelihoods} * \Bigg( \Pr(I_{k} = 1 \mid I_{k-1} = 0, v_{k-1}) * \int U(a, b) * 
p(I_{k-1} = 0, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1} +
\Pr(I_{k} = 1 \mid I_{k-1} = 1, v_{k-1}) * \int N(x_{k-1}, \sigma) * 
p(I_{k-1} = 1, x_{k-1} \mid y_{1:k-1}, \Delta N_{1:k-1}^{1:C}, v_{1:k-1}) * dx_{k-1} \Bigg)
\end{align*}$$